# I am sharing my notebook as It is a research competition, I would love if someone would be able to achieve great with the help of my simple notebook.

I want to give credits to this [notebook](https://www.kaggle.com/soham1024/visualization-using-nilearn) for helping me out in some visuals. 
Feel free to use my notebook and please I request all of you to share your notebooks too.

# IMPORTING MODULES

In [ ]:
import h5py

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, KFold
import scipy as sp
import random
import nilearn as nl
from nilearn import datasets
from nilearn import plotting
from nilearn import image
import nibabel as nib
import nilearn.plotting as nlplt
import matplotlib.pyplot as plt
import seaborn as sns

from functools import partial

from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold
from sklearn import preprocessing
import category_encoders as ce
from sklearn.metrics import mean_squared_error

import torch

import lightgbm as lgb
from glob import glob

import matplotlib.pyplot as plt
%matplotlib inline

# Reading data

In [ ]:
train = pd.read_csv('/kaggle/input/trends-assessment-prediction/train_scores.csv').sort_values(by='Id')

loadings = pd.read_csv('/kaggle/input/trends-assessment-prediction/loading.csv')

fnc = pd.read_csv('/kaggle/input/trends-assessment-prediction/fnc.csv')

sample = pd.read_csv('/kaggle/input/trends-assessment-prediction/sample_submission.csv')

reveal = pd.read_csv('/kaggle/input/trends-assessment-prediction/reveal_ID_site2.csv')

ICN = pd.read_csv('/kaggle/input/trends-assessment-prediction/ICN_numbers.csv')



In [ ]:
!wget https://github.com/Chaogan-Yan/DPABI/raw/master/Templates/ch2better.nii

# *IMPORTANT VISUALS OF BRAIN*

In [ ]:
mat = h5py.File('/kaggle/input/trends-assessment-prediction/fMRI_train/10031.mat','r')
mat.keys()

In [ ]:
sample = mat['SM_feature']


In [ ]:
array = sample[()]
array.shape

In [ ]:
print(array.min(),array.max(),array.mean())

In [ ]:
mat, ax = plt.subplots(1,4)
mat.set_size_inches(25, 10)
for i in range(4):
    Temp = array[i*10, :, 10, :] !=0  
    ax[i].imshow(Temp)
plt.show()

In [ ]:
motor_images = datasets.fetch_neurovault_motor_task()
img = motor_images.images[0]

In [ ]:
nii_loc = "/kaggle/input/trends-assessment-prediction/fMRI_mask.nii"
nii_loc2 = "/kaggle/input/trends-assessment-prediction/fMRI_train/10031.mat"
niiplot = plotting.plot_glass_brain(img)
niiplot

In [ ]:

maskni = nl.image.load_img(nii_loc)
subjectimage = nl.image.new_img_like(nii_loc, array, affine=maskni.affine, copy_header=True)


In [ ]:
smri = 'ch2better.nii'
num_components = subjectimage.shape[-1]

In [ ]:
grid_size = int(np.ceil(np.sqrt(num_components)))
fig, axes = plt.subplots(grid_size, grid_size, figsize=(grid_size*10, grid_size*10))
[axi.set_axis_off() for axi in axes.ravel()]
row = -1
for i, cur_img in enumerate(nl.image.iter_img(subjectimage)):
    col = i % grid_size
    if col == 0:
        row += 1
    nlplt.plot_stat_map(cur_img, bg_img=smri, title="IC %d" % i, axes=axes[row, col], threshold=3, colorbar=False)

# Data Cleansing  

In [ ]:
train.isnull().sum()

In [ ]:
reveal.head

In [ ]:
ICN.head()

In [ ]:
fnc.head()

In [ ]:
train.head()

In [ ]:
train_ids = sorted(loadings[loadings['Id'].isin(train.Id)]['Id'].values)
test_ids = sorted(loadings[~loadings['Id'].isin(train.Id)]['Id'].values)
predictions = pd.DataFrame(test_ids, columns=['Id'], dtype=str)
features = ('age', 'domain1_var1', 'domain1_var2','domain2_var1','domain2_var2')

In [ ]:
data = pd.merge(loadings, train, on='Id').dropna()
X_train = data.drop(list(features), axis=1).drop('Id', axis=1)
y_train = data[list(features)]
X_test = loadings[loadings.Id.isin(test_ids)].drop('Id', axis=1)

# USING MODEL FOR PREDICTIONS

In [ ]:
model = RandomForestRegressor(
    max_depth=10,
    min_samples_split=10,
    min_samples_leaf=5
)
cv = KFold(n_splits = 5, shuffle=True, random_state=29)
grid = {
    'n_estimators':[5,10,20,100]
}
gs = GridSearchCV(model, grid, n_jobs=-1, cv=cv, verbose=1, scoring='neg_mean_absolute_error')

In [ ]:
best_models = {}
for col in features:
    gs.fit(X_train, y_train[col])
    best_models[col] = gs.best_estimator_
    print(gs.best_score_)

In [ ]:
for col in features:
    predictions[col] = best_models[col].predict(X_test)

In [ ]:
def make_sub(predictions):
    features = ('age', 'domain1_var1', 'domain1_var2','domain2_var1','domain2_var2')
    _columns = (0,1,2,3,4)
    tests = predictions.rename(columns=dict(zip(features, _columns)))
    tests = tests.melt(id_vars='Id',value_vars=_columns,value_name='Predicted')
    tests['target'] = tests.variable.map(dict(zip(_columns, features)))
    tests['Id_'] = tests[['Id', 'target']].apply(lambda x: '_'.join((str(x[0]), str(x[1]))), axis=1)
  
    return tests.sort_values(by=['Id', 'variable'])\
              .drop(['Id', 'variable', 'target'],axis=1)\
              .rename(columns={'Id_':'Id'})\
              .reset_index(drop=True)\
              [['Id', 'Predicted']]

In [ ]:
sub = make_sub(predictions)

In [ ]:
sub.head()

In [ ]:
sub.to_csv('firsttry.csv', index=False)

# *THANK YOU.Its just the beginning,I will try my best to improve it and provide better notebook for other. Please upvote if you guys liked it,It would give a  motivation to continue on my work.*